In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium.common.exceptions import NoSuchElementException   
import progressbar

options = Options()
options.headless = True
options.add_argument("--window-size=1920,20200") #this is important, to tell it how much of the webpage to import
driver = webdriver.Chrome(options=options, executable_path=r'/usr/local/bin/chromedriver')

In [68]:
def formazioni(competizione = 'campionato'):
    #if competizione == 'campionato':
    link = 'https://leghe.fantacalcio.it/fantapalla-forever/formazioni?id=185855'
    driver.get(link)

    players = {}
    for l in [2,3,4,5]:
        for k in [1,2]:
            name = driver.find_element_by_xpath("/html/body/div[7]/main/div[3]/div[2]/div[1]/div[1]/div/div/div["+str(l)+"]/div[1]/div["+str(k)+"]/div/div[2]/h4").text


            all_playersE = driver.find_elements_by_xpath("/html/body/div[7]/main/div[3]/div[2]/div[1]/div[1]/div/div/div["+str(l)+"]/div[2]/div["+str(k)+"]/table[1]/tbody/tr[@class='player-list-item even  ']/td[1]/span/span[2]/a")
            all_playersO = driver.find_elements_by_xpath("/html/body/div[7]/main/div[3]/div[2]/div[1]/div[1]/div/div/div["+str(l)+"]/div[2]/div["+str(k)+"]/table[1]/tbody/tr[@class='player-list-item odd  ']/td[1]/span/span[2]/a")
            all_playersEP = driver.find_elements_by_xpath("/html/body/div[7]/main/div[3]/div[2]/div[1]/div[1]/div/div/div["+str(l)+"]/div[2]/div["+str(k)+"]/table[2]/tbody/tr[@class='player-list-item even  ']/td[1]/span/span[2]/a")
            all_playersOP = driver.find_elements_by_xpath("/html/body/div[7]/main/div[3]/div[2]/div[1]/div[1]/div/div/div["+str(l)+"]/div[2]/div["+str(k)+"]/table[2]/tbody/tr[@class='player-list-item odd  ']/td[1]/span/span[2]/a")
            #all_playersE = driver.find_elements_by_xpath("")

            all_pl = all_playersE + all_playersO + all_playersEP + all_playersOP
            names = []
            for pl in all_pl:
                names.append((pl.text).upper())
            players[name] = names
    return pd.DataFrame.from_dict(data=players, orient='index').T

In [3]:
link = 'https://leghe.fantacalcio.it/fantapalla-forever/area-gioco/rose'
    
driver.get(link)
rose = {}


for j in range(1,9):
    test_1 = []
    name = (driver.find_element_by_xpath('/html/body/div[7]/main/div[3]/div[2]/div[1]/div[2]/div[2]/ul/li['+str(j)+']/div/div[1]/div[2]/h4').text).upper()
    players = driver.find_elements_by_xpath('/html/body/div[7]/main/div[3]/div[2]/div[1]/div[2]/div[2]/ul/li['+str(j)+']/table/tbody/tr[@class]/td[2]/a/b')
    for pl in players:
        test_1.append((pl.text).upper())

    rose[name] = test_1

In [21]:
def rose(link = 'https://leghe.fantacalcio.it/fantapalla-forever/area-gioco/rose'):
    
    driver.get(link)
    rose = {}


    for j in range(1,9):
        test_1 = []
        name = (driver.find_element_by_xpath('/html/body/div[7]/main/div[3]/div[2]/div[1]/div[2]/div[2]/ul/li['+str(j)+']/div/div[1]/div[2]/h4').text).upper()
        players = driver.find_elements_by_xpath('/html/body/div[7]/main/div[3]/div[2]/div[1]/div[2]/div[2]/ul/li['+str(j)+']/table/tbody/tr[@class]/td[2]/a/b')
        for pl in players:
            test_1.append((pl.text).upper())

        rose[name] = test_1
        
        
    return pd.DataFrame.from_dict(data=rose, orient='index').T

In [16]:
def infortunati(link = 'https://www.pianetafanta.it/Giocatori-Infortunati.asp'):

    driver.get(link)
    names = driver.find_elements_by_xpath("//*[@id='my_id_div']/div[6]/div[11]/div[2]/div/div[@class]/div/table/tbody/tr[@class]/td[2]/a/strong")

    players = []
    for n in names:
        players.append(n.text.upper())

    return players

In [22]:
I=infortunati()

In [23]:
R=rose()

In [50]:
def count_inf(I , R ):
    count = {}
    #test = list(filter(None,I.values.flatten()))
    #separator = ','
    all_names = ''.join(I)
    for team in R:
        c = 0
        for player in R[team]:
            if player == None:
                continue
            if player in ''.join(I):
                c+=1
        count[team] = c
    return pd.DataFrame(data = count, index = ['tot Infortunati'])

In [51]:
count_inf(I,R)

,XYZ,MAINZ NA GIOIA,I DISEREDATI,PALLA PAZZA,AS 800A,PDG 1908,SOROS FC,IGNORANZA EVERYWHERE
tot Infortunati,14,8,6,3,5,7,9,8


In [7]:
import time

In [67]:
def infortunati(link = 'https://www.fantacalcio.it/cartella-medica/'):

    driver.get(link)
    button = driver.find_element_by_id("tabAll")

    driver.execute_script("arguments[0].click();", button)

    test = driver.find_elements_by_xpath("/html/body/div[7]/div[5]/main/div/div/div/div[1]/div[2]/div[5]/div[@class]/div/div/div[2]/div/div[1]/p[@class]/span")
    all_inf = []
    for pl in test:
        all_inf.append(pl.text)
    return all_inf

In [68]:
infortunati()

['CARNESECCHI:',
 'GOLLINI:',
 'CALDARA:',
 'MONCINI:',
 'BARBA:',
 'VIOLA:',
 'DIJKS:',
 'MEDEL:',
 'POLI:',
 'SKOV OLSEN:',
 'PEREIRO:',
 'BENALI:',
 'RIVIERE:',
 '-',
 'LERAGER:',
 'ZAPPACOSTA:',
 'DESTRO:',
 'MALES:',
 'CRISCITO:',
 'CASSATA:',
 'NAINGGOLAN:',
 'RADU I.:',
 'VECINO:',
 'SKRINIAR:',
 'YOUNG:',
 'GAGLIARDINI:',
 'MCKENNIE:',
 'RAMSEY:',
 'RONALDO:',
 'DE LIGT:',
 'PEREIRA A.:',
 'PROTO:',
 'LUIZ FELIPE:',
 'RADU:',
 'LAZZARI:',
 'LULIC:',
 'DUARTE:',
 'GABBIA:',
 'MUSACCHIO:',
 'REBIC:',
 'ELMAS:',
 'ZIELINSKI:',
 'INSIGNE:',
 'INGLESE:',
 'SPROCATI:',
 'BRUNETTA:',
 'MIHAILA:',
 'DEZI:',
 'BUSI:',
 'VALENTI:',
 'CORNELIUS:',
 'DIAWARA:',
 'PASTORE:',
 'ZANIOLO:',
 'SMALLING:',
 'GABBIADINI:',
 'AYHAN:',
 'ROGERIO:',
 'ROMAGNA:',
 'TOLJAN:',
 'DEIOLA:',
 'ISMAJLI:',
 'CAPRADOSSI:',
 'RAMOS:',
 'MATTIELLO:',
 'ZOET:',
 'MARCHIZZA:',
 'MASTINU:',
 'GALABINOV:',
 'BASELLI:',
 'MANDRAGORA:',
 'WALACE:',
 'MUSSO:',
 'STRYGER LARSEN:',
 'JAJALO:',
 'DANZI:',
 'CETIN:',
 'B